In [1]:
import os
import time
import torch
import warnings
import numpy as np
import nibabel as nib
from time import strftime
from functools import partial
from copy import deepcopy
from torch.utils.data import DataLoader
import torch.nn.functional as F
from captum.attr import IntegratedGradients, InputXGradient, DeepLift
from captum.attr import Deconvolution, Occlusion
from captum.attr import FeatureAblation, FeaturePermutation
from captum.attr import GuidedBackprop, GuidedGradCam, LayerGradCam
from captum.attr import DeepLiftShap, GradientShap, KernelShap
from models import SimpleResNet
from datasets import ShapesDataset, LocationDataset, RotationDataset, ScaleDataset
from datasets import ContrastDataset
from utils import color_codes, time_to_string

warnings.simplefilter("ignore")

master_seed = 42
data_folder = '/home/mariano/data/Saliency/'

def attribution(x, y, attr_m, *args, **kwargs):
    attr = attr_m.attribute(
        x, target=0, *args, **kwargs
    )
    if x.shape != attr.shape:
        attr = F.interpolate(attr, size=x.size()[2:], mode='trilinear')
    attr_map = attr.squeeze().detach().cpu().numpy()
    if y != 1:
        attr_map = - attr_map
    return attr_map
    
def run_experiments(
    master_seed, property_name, dataset, network, weight_path,
    test_samples=100, train_samples=10000, val_samples=1000,
    im_size=(64, 64, 64), n_seeds=30
):
    np.random.seed(master_seed)
    seeds = np.random.randint(0, 100000, n_seeds)
    c = color_codes()
    saliency_path = os.path.join(weight_path, property_name.capitalize())
    if not os.path.exists(saliency_path):
        os.mkdir(saliency_path)
    print(
        '{:}[{:}] {:}Creating {:}testing{:} dataset '
        'for {:} classification{:}'.format(
            c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['g'], c['nc'] + c['y'],
            c['nc'] + c['g'], c['b'] + property_name + c['nc'] + c['g'], c['nc']
        )
    )
    testing_set = dataset(im_size=im_size, n_samples=test_samples, seed=master_seed)
    train_batch = 20
    test_batch = 50
    epochs = 2
    patience = 2
    for test_n, seed in enumerate(seeds):
        acc = 0
        print(
            '{:}[{:}] {:}Starting experiment {:}(seed {:05d}){:} [{:02d}/{:02d}] '
            '{:}for {:} classification{:}'.format(
                c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['g'],
                c['nc'] + c['y'], seed, c['nc'] + c['c'], test_n + 1, len(seeds),
                c['nc'] + c['g'], c['b'] + property_name + c['nc'] + c['g'], c['nc']
            )
        )
        np.random.seed(seed)
        torch.manual_seed(seed)
        net = network(n_images=1)
        
        methods = [
            # 'Input'-based
            (IntegratedGradients, 'IntegratedGradients'),
            (InputXGradient, 'InputXGradient'),
            (DeepLift, 'DeepLift'),
            # Inverse
            (Deconvolution, 'Deconvolution'),
            # Perturbation-based
            # (partial(Occlusion, sliding_window_shapes=(8, 8, 8)), 'Occlusion'),
            # (FeaturePermutation, 'FeaturePermutation'),
            # GradCAM-related
            (partial(LayerGradCam, layer=net.extractor.down[1].conv), 'LayerGradCam'),
            (partial(GuidedGradCam, layer=net.extractor.down[1].conv), 'GuidedGradCam'),
            (GuidedBackprop, 'GuidedBackprop'),
        ]
               
        net.init = False
        n_param = sum(
            p.numel() for p in net.parameters() if p.requires_grad
        )
        training_set = dataset(im_size=im_size, n_samples=train_samples)
        validation_set = dataset(im_size=im_size, n_samples=val_samples)
        training_loader = DataLoader(
            training_set, train_batch, True
        )
        validation_loader = DataLoader(
            validation_set, test_batch
        )
        model_path = os.path.join(
            weight_path, '{:}_s{:05d}.pt'.format(property_name, seed)
        )
        try:
            net.load_model(model_path)
        except IOError:
            net.fit(training_loader, validation_loader, epochs=epochs, patience=patience)
            net.save_model(model_path)
            
        print(
            '{:}[{:}] {:}Testing {:}(seed {:05d}){:} [{:02d}/{:02d}] '
            '{:}for {:} classification <{:04d}/{:04d} samples>{:}'.format(
                c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['g'],
                c['nc'] + c['y'], seed, c['clr'] + c['c'], test_n + 1, len(seeds),
                c['nc'] + c['g'], c['b'] + property_name + c['nc'] + c['g'],
                len(training_set), len(validation_set), c['nc']
            )
        )

        for i, (x, (y, mask)) in enumerate(DataLoader(testing_set, test_batch)):
            pred_y = net.inference(x.numpy(), False)
            pred_y = (pred_y > 0).squeeze().astype(np.uint8)
            y = y.numpy().astype(np.uint8)
            acc += np.sum(y == pred_y) / len(testing_set)
        print(
            '{:}[{:}] {:}Accuracy {:}(seed {:05d}){:} [{:02d}/{:02d}] {:}'
            '{:5.3f}{:}'.format(
                c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['g'],
                c['nc'] + c['y'], seed, c['clr'] + c['c'], test_n + 1, len(seeds),
                c['nc'] + c['b'], acc, c['nc']
            )
        )
    print(
        '{:}[{:}] {:}Saliency maps{:}'.format(
            c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['g'], c['nc']
        )
    )
    
    init_start = time.time()
    for i, (x, (y, mask)) in enumerate(DataLoader(testing_set, 1)):
        y = y.numpy().astype(np.uint8)
        x_cuda = x.to(net.device)
        x_cuda.requires_grad_()
        # Image
        filepath = os.path.join(
            saliency_path, '{:}-{:04d}_s{:05d}_c{:d}_image.nii.gz'.format(
                property_name, i, master_seed, y.tolist()[0]
            )
        )
        nii = nib.Nifti1Image(np.squeeze(x.numpy()), np.eye(4))
        nii.to_filename(filepath)
        filepath = os.path.join(
            saliency_path, '{:}-{:04d}_s{:05d}_c{:d}_mask.nii.gz'.format(
                property_name, i, master_seed, y.tolist()[0]
            )
        )
        nii = nib.Nifti1Image(np.squeeze(mask.numpy()).astype(np.uint8), np.eye(4))
        nii.to_filename(filepath)
        for attr_m, attr_name in methods:
            map_path = os.path.join(
                saliency_path, '{:}-{:04d}_{:}_c{:d}.nii.gz'.format(
                    property_name, i, attr_name, y.tolist()[0]
                )
            )
            try:
                nib.load(map_path).get_fdata()
            except IOError:
                maps = []
                predictions = []
                for test_n, seed in enumerate(seeds):
                    time_elapsed = time.time() - init_start
                    eta = (len(testing_set) - (i + 1)) * time_elapsed / (i + 1)
                    model_path = os.path.join(
                        weight_path, '{:}_s{:05d}.pt'.format(property_name, seed)
                    )
                    net.load_model(model_path)
                    net.eval()
                    print(' '.join([' '] * 300), end='\r')
                    print(
                        '\033[KGenerating {:} map'
                        '(case {:d}/{:d} | seed {:05d} [{:02d}/{:02d}]) {:} ETA {:}'.format(
                            attr_name, i + 1, len(testing_set),
                            seed, test_n, len(seeds),
                            time_to_string(time_elapsed),
                            time_to_string(eta),
                        ), end='\r'
                    )
                    maps.append(
                        attribution(x_cuda, y, attr_m(net))
                    )
                    with torch.no_grad():
                        predictions.append(
                            int(net(x_cuda).squeeze().cpu().detach().numpy().tolist() >= 0)
                        )
                hdr = deepcopy(nii.header)
                hdr['descrip'] = str.encode('Predictions: {:}'.format(
                    ' '.join([
                        str(p) for p in predictions
                    ])
                ))
                out_nii = nib.Nifti1Image(np.stack(maps, axis=-1), None, hdr)
                out_nii.to_filename(map_path)
            

    print(
        '{:}[{:}] {:}Experiments for {:} classification finished{:}'.format(
            c['clr'] + c['c'], strftime("%m/%d/%Y - %H:%M:%S"), c['r'], c['nc'] + c['y'],
            c['nc'] + c['r'], c['b'] + property_name + c['nc'] + c['r'], c['nc']
        )
    )

In [2]:
run_experiments(
    master_seed, 'shape', ShapesDataset, SimpleResNet, data_folder
)

[04/21/2023 - 09:39:45] Creating testing dataset for shape classification
[04/21/2023 - 09:39:53] Starting experiment (seed 15795) [01/30] for shape classification                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
[04/21/2023 - 09:39:57] Testing (seed 15795) [01/30] for shape classification <20000/2000 samples>
[04/21/2023 - 09:39:58] Accuracy (seed 15795) [01/30] 1.000
[04/21/2023 - 09:39:58] Starting experiment (seed 00860) [02/30] for shape classification
[04/21/2023 - 09:39:58] Testing (seed 00860) [02/30] for shape classification <20000/2000 samples>
[04/21/2023 - 09:39:59] Accuracy (s

[04/21/2023 - 09:40:18] Accuracy (seed 05311) [20/30] 1.000
[04/21/2023 - 09:40:18] Starting experiment (seed 83104) [21/30] for shape classification
[04/21/2023 - 09:40:18] Testing (seed 83104) [21/30] for shape classification <20000/2000 samples>
[04/21/2023 - 09:40:19] Accuracy (seed 83104) [21/30] 1.000
[04/21/2023 - 09:40:19] Starting experiment (seed 53707) [22/30] for shape classification
[04/21/2023 - 09:40:19] Testing (seed 53707) [22/30] for shape classification <20000/2000 samples>
[04/21/2023 - 09:40:20] Accuracy (seed 53707) [22/30] 1.000
[04/21/2023 - 09:40:20] Starting experiment (seed 85305) [23/30] for shape classification
[04/21/2023 - 09:40:20] Testing (seed 85305) [23/30] for shape classification <20000/2000 samples>
[04/21/2023 - 09:40:21] Accuracy (seed 85305) [23/30] 1.000
[04/21/2023 - 09:40:21] Starting experiment (seed 28693) [24/30] for shape classification
[04/21/2023 - 09:40:21] Testing (seed 28693) [24/30] for shape classification <20000/2000 samples>
[04/

In [3]:
run_experiments(
    master_seed, 'location', LocationDataset, SimpleResNet, data_folder
)

[04/21/2023 - 09:41:37] Creating testing dataset for location classification
[04/21/2023 - 09:41:45] Starting experiment (seed 15795) [01/30] for location classification                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
[04/21/2023 - 09:41:45] Testing (seed 15795) [01/30] for location classification <20000/2000 samples>
[04/21/2023 - 09:41:46] Accuracy (seed 15795) [01/30] 1.000
[04/21/2023 - 09:41:46] Starting experiment (seed 00860) [02/30] for location classification
[04/21/2023 - 09:41:46] Testing (seed 00860) [02/30] for location classification <20000/2000 samples>
[04/21/2023 - 09:41:47]

[04/21/2023 - 09:42:05] Accuracy (seed 67969) [19/30] 1.000
[04/21/2023 - 09:42:05] Starting experiment (seed 05311) [20/30] for location classification
[04/21/2023 - 09:42:05] Testing (seed 05311) [20/30] for location classification <20000/2000 samples>
[04/21/2023 - 09:42:06] Accuracy (seed 05311) [20/30] 1.000
[04/21/2023 - 09:42:06] Starting experiment (seed 83104) [21/30] for location classification
[04/21/2023 - 09:42:06] Testing (seed 83104) [21/30] for location classification <20000/2000 samples>
[04/21/2023 - 09:42:07] Accuracy (seed 83104) [21/30] 1.000
[04/21/2023 - 09:42:07] Starting experiment (seed 53707) [22/30] for location classification
[04/21/2023 - 09:42:07] Testing (seed 53707) [22/30] for location classification <20000/2000 samples>
[04/21/2023 - 09:42:08] Accuracy (seed 53707) [22/30] 1.000
[04/21/2023 - 09:42:08] Starting experiment (seed 85305) [23/30] for location classification
[04/21/2023 - 09:42:09] Testing (seed 85305) [23/30] for location classification <

In [4]:
run_experiments(
    master_seed, 'rotation', RotationDataset, SimpleResNet, data_folder
)

[04/21/2023 - 09:43:39] Creating testing dataset for rotation classification
[04/21/2023 - 09:43:48] Starting experiment (seed 15795) [01/30] for rotation classification                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
[04/21/2023 - 09:43:48] Testing (seed 15795) [01/30] for rotation classification <20000/2000 samples>
[04/21/2023 - 09:43:49] Accuracy (seed 15795) [01/30] 0.975
[04/21/2023 - 09:43:49] Starting experiment (seed 00860) [02/30] for rotation classification
[04/21/2023 - 09:43:50] Testing (seed 00860) [02/30] for rotation classification <20000/2000 samples>
[04/21/2023 - 09:43:50]

[04/21/2023 - 09:44:09] Accuracy (seed 67969) [19/30] 0.970
[04/21/2023 - 09:44:09] Starting experiment (seed 05311) [20/30] for rotation classification
[04/21/2023 - 09:44:09] Testing (seed 05311) [20/30] for rotation classification <20000/2000 samples>
[04/21/2023 - 09:44:10] Accuracy (seed 05311) [20/30] 0.985
[04/21/2023 - 09:44:10] Starting experiment (seed 83104) [21/30] for rotation classification
[04/21/2023 - 09:44:10] Testing (seed 83104) [21/30] for rotation classification <20000/2000 samples>
[04/21/2023 - 09:44:11] Accuracy (seed 83104) [21/30] 0.980
[04/21/2023 - 09:44:11] Starting experiment (seed 53707) [22/30] for rotation classification
[04/21/2023 - 09:44:11] Testing (seed 53707) [22/30] for rotation classification <20000/2000 samples>
[04/21/2023 - 09:44:12] Accuracy (seed 53707) [22/30] 0.965
[04/21/2023 - 09:44:12] Starting experiment (seed 85305) [23/30] for rotation classification
[04/21/2023 - 09:44:12] Testing (seed 85305) [23/30] for rotation classification <

In [ ]:
run_experiments(
    master_seed, 'scale', ScaleDataset, SimpleResNet, data_folder
)

[04/21/2023 - 09:45:34] Creating testing dataset for scale classification
[04/21/2023 - 09:45:43] Starting experiment (seed 15795) [01/30] for scale classification                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
[04/21/2023 - 09:45:43] Testing (seed 15795) [01/30] for scale classification <20000/2000 samples>
[04/21/2023 - 09:45:44] Accuracy (seed 15795) [01/30] 1.000
[04/21/2023 - 09:45:44] Starting experiment (seed 00860) [02/30] for scale classification
[04/21/2023 - 09:45:44] Testing (seed 00860) [02/30] for scale classification <20000/2000 samples>
[04/21/2023 - 09:45:45] Accuracy (s

           Epoch 001 |  0.0305 |  0.0000 |   0.0178 |   0.0000 |   0.0000 |   0.0035 | 0.000 | 18m 56s                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Training finished in 2 epochs (37m 43s) with minimum loss = 0.000000 (epoch 0)
[04/21/2023 - 10:23:44] Testing (seed 62955) [17/30] for scale classification <20000/2000 samples>
[04/21/2023 - 10:23:45] Accuracy (seed 62955) [17/30] 0.990
[04/21/2023 - 10:23:45] Starting experiment (seed 64925) [18/30] for scale classification
           Epoch num |  train  |   val   |   xent   |    fn    |    fp    |   acc    |  drp  |
       

[04/21/2023 - 12:51:47] Accuracy (seed 83104) [21/30] 0.995
[04/21/2023 - 12:51:47] Starting experiment (seed 53707) [22/30] for scale classification
           Epoch num |  train  |   val   |   xent   |    fn    |    fp    |   acc    |  drp  |
           ----------|---------|---------|----------|----------|----------|----------|-------|
           Epoch 000 |  0.0500 |  0.0000 |   0.0220 |   0.0000 |   0.0000 |   0.0060 | 0.000 | 18m 25s                                                                                                                                                                                                                                                                                                                                                                                                                                               
           Epoch 001 |  0.0273 |  0.0000 |   0.0213 |   0.0000 |   0.0000 |   0.0065 | 0.000 | 18m 24s                        

           Epoch 000 |  0.0431 |  0.0000 |   0.0512 |   0.0000 |   0.0000 |   0.0105 | 0.000 | 18m 31s                                                                                                                                                                                                                                                                                                                                                                                                                                               


In [ ]:
run_experiments(
    master_seed, 'contrast', ContrastDataset, SimpleResNet, data_folder
)